# PUMC Course Select

北京协和医学院研究生选课脚本

## Step 1

Prepare your environment. You'll need:

1. Install Python deps. The simplest way is to run `uv sync`.
2. Install [Tesseract](https://github.com/tesseract-ocr/tesseract).

In [1]:
import re
import cv2
import sys
import time
import random
import numexpr
import requests
import pytesseract
import numpy as np
from PIL import Image
from io import BytesIO
from datetime import datetime
from IPython.display import display

## Step 2

Initiate a login process.

In [2]:
base_url = 'https://graduatexk.pumc.edu.cn/graduate'

login_url        = f'{base_url}/index.do'
captcha_url      = f'{base_url}/getCaptcha.do'
bulletin_url     = f'{base_url}/listMyBulletined.do'
login_submit_url = f'{base_url}/j_acegi_security_check'
logout_url       = f'{base_url}/sso/sso_logout.jsp'
elect_course_url = f'{base_url}/stuelectcourse/addScoreFromPlan.do'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'}

In [4]:
session = requests.Session()

In [5]:
session.headers.update(headers)
session.headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [6]:
login = session.get(login_url)

> The login form requires a `token` field from here.

In [7]:
token_start_pattern = '<input type="hidden" name="token" value="'
token_start = login.text.find(token_start_pattern)
token_end = login.text.find('"/>', token_start)

In [8]:
token = login.text[token_start + len(token_start_pattern):token_end]
token

'907087483052728308617531836351'

### Step 2.1

Solve the captcha.

In [9]:
captcha = session.get(captcha_url)

#### Details

Here's the comparison of three methods.

No need to run in the login process.

In [10]:
img = Image.open(BytesIO(captcha.content))
img

In [11]:
np_array = np.frombuffer(captcha.content, dtype=np.uint8)
cv2_image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
cv2_image_crop = cv2_image[:, :72]

> Method 1

In [12]:
gray = cv2.cvtColor(cv2_image_crop, cv2.COLOR_BGR2GRAY)
gray_thr = cv2.threshold(gray, 0, 256,
                      cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

Image.fromarray(cv2.cvtColor(gray_thr, cv2.COLOR_BGR2RGB))

> Method 2

In [13]:
hsv = cv2.cvtColor(cv2_image_crop, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
mask = ((s > 30) | (v < 230)).astype(np.uint8) * 255

# Optional cleanup
kernel = np.ones((2, 2), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

img = cv2.bitwise_not(mask)
Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

> Method 3 (best)

In [14]:
gray = cv2.cvtColor(cv2_image_crop, cv2.COLOR_BGR2GRAY)
gray_thr = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    15, 4
)
img = cv2.bitwise_not(gray_thr)
Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [15]:
text = pytesseract.image_to_string(
    img,
    config="--psm 7 -c tessedit_char_whitelist=0123456789+-="
)

expr = re.findall(r"[0-9+\-]+", text)

In [16]:
expr = expr[0]
expr

'49-44'

> "+" is very likely to be recognized as "4"

In [17]:
if expr[int(len(expr) / 2)] == '4':
    expr = expr[:int(len(expr) / 2)] + '+' + expr[int(len(expr) / 2) + 1:]
elif len(expr) % 2 == 0 and expr[int(len(expr) / 2) + 1] == '4':
    expr = expr[:int(len(expr) / 2) + 1] + '+' + expr[int(len(expr) / 2) + 2:]

expr

'49-44'

In [20]:
eval(expr)

5

> Here's a safer way, not necessary though

In [21]:
captcha_value = int(numexpr.evaluate(expr))
captcha_value

5

#### Entry

In [22]:
def solve_captcha(s: requests.Session, debug: bool = False) -> int:
    try:
        captcha = session.get(captcha_url)
    
        if debug:
            display(Image.open(BytesIO(captcha.content)))
    
        np_array = np.frombuffer(captcha.content, dtype=np.uint8)
        cv2_image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
        cv2_image_crop = cv2_image[:, :72]
        
        gray = cv2.cvtColor(cv2_image_crop, cv2.COLOR_BGR2GRAY)
        gray_thr = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            15, 4
        )
        img = cv2.bitwise_not(gray_thr)
    
        if debug:
            display(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
    
        text = pytesseract.image_to_string(
            img,
            config="--psm 7 -c tessedit_char_whitelist=0123456789+-"
        )
        if debug:
            print(f'OCR result: {text.strip()}')
        expr = re.findall(r"[0-9]+[4+\-][0-9]+", text)
        expr = expr[0]
        if debug:
            print(f'Regex result: {expr}')
        
    
        if expr[int(len(expr) / 2)] == '4':
            expr = expr[:int(len(expr) / 2)] + '+' + expr[int(len(expr) / 2) + 1:]
        elif len(expr) % 2 == 0 and expr[int(len(expr) / 2) + 1] == '4':
            expr = expr[:int(len(expr) / 2) + 1] + '+' + expr[int(len(expr) / 2) + 2:]
    
        if debug:
            print(f'Final formula: {expr}')
        captcha_value = int(numexpr.evaluate(expr))
        assert captcha_value > 0
        return captcha_value
    except IndexError:
        print('OCR error?')
        return 0
    except AssertionError:
        print('OCR error!')
        return 0
    except SyntaxError:
        print('OCR error!')
        return 0

In [25]:
captcha_value = solve_captcha(session, debug=True)
captcha_value

OCR result: 46442
Regex result: 46442
Final formula: 46+42


88

### Step 2.2

Login.

If logged in, the bulletin will no be empty, a `classicLook0` class will appear.

In [27]:
def check_login_status(s: requests.Session) -> bool:
    bulletin = s.get(bulletin_url)
    return bulletin.status_code == 200 and 'classicLook0' in bulletin.text

In [28]:
check_login_status(session)

False

**Fill your username and [SM3](https://en.wikipedia.org/wiki/SM3_(hash_function)) hashed password:**

The simplest way is to use the browser console on the login page.

You may also try install one of the Python liberies providing this.

<img src="assets/sm3.webp" width="50%" />

In [29]:
username = 'username'
password_sm3 = '08594e140bcc046e345325435218f67a85c38c63de6443b197b544d70ee62f26'

In [32]:
payload = {
    'token': token,
    'j_username': username,
    'j_password': password_sm3,
    'j_captcha': captcha_value,
    'groupId': ''
}

In [33]:
session.post(login_submit_url, data=payload)

<Response [200]>

In [34]:
check_login_status(session)

True

Solve the captcha recursively until login successfully.

In [38]:
if not check_login_status(session):
    login = session.get(login_url)
    token_start_pattern = '<input type="hidden" name="token" value="'
    token_start = login.text.find(token_start_pattern)
    token_end = login.text.find('"/>', token_start)
    token = login.text[token_start + len(token_start_pattern):token_end]
    token
    
    while not check_login_status(session):
        time.sleep(1 + random.random())
        session.post(
            login_submit_url,
            data={
                'token': token,
                'j_username': username,
                'j_password': password_sm3,
                'j_captcha': solve_captcha(session, debug=True),
                'groupId': ''
            }
        )

OCR result: 37427
Regex result: 37427
Final formula: 37+27


### Step 3

Select course

Find your course id (`taskid`) by inspecting the + button .

<img src="assets/taskid.webp" width="50%" />

In [40]:
course_ids = ['114514']

> not chosen in plan

In [41]:
result = session.get(elect_course_url, params={'taskid': course_ids[0]}).text
result

'{\n  "message": "该课程不在培养计划中",\n  "status": "1"\n}'

> normal status

In [42]:
result = session.get(elect_course_url, params={'taskid': course_ids[0]}).text
result

'{\n  "message": "选课人数超过课容量",\n  "status": "1"\n}'

### Step 4

Take a coffee.

In [16]:
while '超过课容量' in result:
    for course_id in course_ids:
        response = session.get(elect_course_url, params={'taskid': course_id})
        result = response.text
        sys.stdout.write('\r' + when() + '\t' + result.replace('\n', ''))
        time.sleep(5 + 5*random.random())

12:38:42	{  "scoreid": 114514,  "lotstatus": "选中",  "credit": 2.0,  "status": "0"}

> The logic here is to stop if any course is chosen. You may do some modification to make it stop only if all courses are chosen.

> Not necessary, but you can logout afterwards.

In [43]:
logout = session.get(logout_url)

In [44]:
check_login_status(session)

False